In [1]:
import torch
from torch import nn

from src.data_loading import wdl_to_cp
from src.data_loading_halfkp import HalfKpDataset, M

from src.core import train, test

In [2]:
BATCH_SIZE = 64

In [3]:
from src.patches import TRAIN_DATASET_PATCH, TEST_DATASET_PATCH

train_dataset = HalfKpDataset(TRAIN_DATASET_PATCH, BATCH_SIZE)
test_dataset = HalfKpDataset(TEST_DATASET_PATCH, BATCH_SIZE)
len(train_dataset), len(test_dataset)

(13149, 3287)

In [6]:
def accuracy(out, truth):
    return wdl_to_cp(torch.abs(truth - out))

In [7]:
x0 = 2 * M
x1 = 2 ** 8
x2 = 2 ** 5


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(x0, x1)
        self.layer2 = nn.Linear(x1, x2)
        self.layer3 = nn.Linear(x2, 1)
        self.classifier = nn.Sequential(self.layer1,
                                        nn.ReLU(),
                                        self.layer2,
                                        nn.ReLU(),
                                        self.layer3)

    def forward(self, X):
        return self.classifier.forward(X)


model = Model()

In [ ]:
test(test_dataset, BATCH_SIZE, model, nn.MSELoss(), accuracy)

Dataset size: 3287


In [ ]:
test(train_dataset, BATCH_SIZE, model, nn.MSELoss(), accuracy)

In [ ]:
train(train_dataset,
      test_dataset,
      BATCH_SIZE,
      model,
      nn.MSELoss(),
      torch.optim.SGD(model.classifier.parameters(), lr=0.001),
      accuracy,
      300)

In [ ]:
test(test_dataset, BATCH_SIZE, model, nn.MSELoss(), accuracy)

In [ ]:
test(train_dataset, BATCH_SIZE, model, nn.MSELoss(), accuracy)

In [ ]:
torch.save(dict(model.state_dict()), 'model.pt')